In [6]:
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import os

def process_csv_to_image(csv_file, output_folder):
    # Read the CSV
    df = pd.read_csv(csv_file)

    # Extract page dimensions
    page_dimensions = df['mask_shape'].iloc[0]
    page_dimensions = page_dimensions.strip('()').split(',')
    page_height, page_width = int(page_dimensions[0]), int(page_dimensions[1])

    # Remove items that don't extend past the first 10% of the page
    left_margin_limit = page_width * 0.1
    df = df[df['x1f'] > left_margin_limit]
    
    # Remove items that aren't primary
    df = df[df['is_primary'] == 1] 

    # Sort by column and then by y0f within each column
    df.sort_values(by=['column', 'y0f'], inplace=True)

    # Initialize variables for tracking the pixel shifts
    shifts = {col: 15 for col in df['column'].unique()}
    column_0_shifts = {}  # To track the shifts applied to column 0 boxes

    # Apply shifts to column 0
    for index, row in df[df['column'] == 0].iterrows():
        df.at[index, 'y0f_shifted'] = df.at[index, 'y0f'] + shifts[0]
        df.at[index, 'y1f_shifted'] = df.at[index, 'y1f'] + shifts[0]
        column_0_shifts[index] = shifts[0]  # Track the shift applied
        shifts[0] += 15

    # Function to find the closest box in column 0 and its shift
    def find_closest_shift_in_column_0(y0f):
        closest_row_index = None
        closest_shift = 0
        min_distance = float('inf')
        for index, row in df[df['column'] == 0].iterrows():
            distance = abs(row['y0f'] - y0f)
            if distance < min_distance:
                closest_row_index = index
                min_distance = distance
                closest_shift = column_0_shifts[index]
        return closest_shift

    # Process non-zero columns
    for column in df['column'].unique():
        if column != 0:
            initial_shift = None
            for index, row in df[df['column'] == column].iterrows():
                if initial_shift is None:
                    # Find the shift for the first box in the column based on its closest box in column 0
                    initial_shift = find_closest_shift_in_column_0(row['y0f']) -15
                df.at[index, 'y0f_shifted'] = df.at[index, 'y0f'] + initial_shift + shifts[column]
                df.at[index, 'y1f_shifted'] = df.at[index, 'y1f'] + initial_shift + shifts[column]
                shifts[column] += 15

    # Adjust the page height
    page_height += max(shifts.values())

    # Create a blank image with the adjusted page height
    image = Image.new("RGB", (page_width, page_height), "white")
    draw = ImageDraw.Draw(image)

    # Define a font for the label
    try:
        font = ImageFont.truetype("arial.ttf", 8)
    except IOError:
        font = ImageFont.load_default()

    # Iterate over each row in the DataFrame to draw boxes and labels
    for _, row in df.iterrows():
        # Draw the bounding box
        box = (row['x0f'], row['y0f_shifted'], row['x1f'], row['y1f_shifted'])
    #     draw.rectangle(box, outline="black")

        # Text to be placed
        text = "json_element: " + str(row['category_lbl'])
        text_size = draw.textsize(text, font=font)

        # Coordinates for the text and its background
        text_x = box[0]
        text_y = box[1] - text_size[1] - 1  # 5 pixels above the box
        background_box = (text_x, text_y, text_x + text_size[0], text_y + text_size[1])

        # Draw the white rectangle and the text
        draw.rectangle(background_box, fill="white")
        draw.text((text_x, text_y), text, fill="black", font=font)

        # Paste the corresponding image
        img_path = os.path.join(folder, row['mask_img_file_names'])
        if os.path.exists(img_path):
            img = Image.open(img_path)
            img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
            image.paste(img, (int(box[0]), int(box[1])))

    # Save the final image
    base_name = os.path.basename(csv_file)
    output_path = os.path.join(output_folder, base_name.replace('_mask_mask_summary.csv', '.png'))
    image.save(output_path)
    return output_path

# Folder paths
folder = r'C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2'
output_folder = r'C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated'  # Folder to save PNGs

# List all CSV files in the folder
csv_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]

# Process each CSV file
for csv_file in csv_files:
    output_png = process_csv_to_image(csv_file, output_folder)
    print(f"Processed: {csv_file} -> {output_png}")


C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(b

Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1503.00841_page_0001_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1503.00841_page_0001.png


C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(b

C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  tex

Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1503.00841_page_0002_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1503.00841_page_0002.png


...

C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(b

Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1601.02403_page_0050_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1601.02403_page_0050.png
Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1601.02403_page_0051_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1601.02403_page_0051.png
Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1601.02403_page_0052_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1601.02403_page_0052.png


C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(b

Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1601.02403_page_0053_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1601.02403_page_0053.png
Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1601.02403_page_0054_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1601.02403_page_0054.png
Processed: C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_masks_2\1601.02403_page_0055_mask_mask_summary.csv -> C:\Users\Shadow\Downloads\qasper_DiT_DLA_Results_11-13-2023\s3_outputs\page_images_annotated\1601.02403_page_0055.png


C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(box[3]) - int(box[1])), Image.ANTIALIAS)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:81: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(text, font=font)
C:\Users\Shadow\AppData\Local\Temp\ipykernel_8240\309682804.py:96: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((int(box[2]) - int(box[0]), int(b